In [1]:
import torch

In [3]:
print(torch.tensor([1,2,3],dtype=torch.float32).numpy())

[1. 2. 3.]


In [5]:
print(torch.rand(2,3))

tensor([[0.5498, 0.8155, 0.6635],
        [0.1120, 0.4679, 0.2431]])


In [8]:
x = torch.tensor(1.0,dtype=torch.float32)
y = torch.tensor(2.0,dtype=torch.float32)
w = torch.tensor(1.0,dtype=torch.float32, requires_grad=True)

y_pred  = x*w
loss = (y_pred-y)**2
loss.backward()
print(w.grad)


tensor(-2.)


#### Gradient descent using numpy

In [10]:
import numpy as np

In [28]:
x = np.array([1,2,3],dtype = np.float32)
y = np.array([3,2,5],dtype = np.float32)
w = np.array([6,2,10],dtype = np.float32)

def forward(x,w):
    return x*w
def loss(y,y_pred):
    return ((y-y_pred)**2).mean()
def gradient(w,y,y_pred):
    return np.dot(2*x,y_pred-y).mean()

In [29]:
learning_rate = 0.01
for i in range(10):
    y_pred = forward(x,w)
    loss_value = loss(y,y_pred)
    print(loss_value)
    gradient_value = gradient(x,y,y_pred)
    w-=gradient_value*learning_rate

212.66667
135.56477
95.595184
74.87495
64.13357
58.565243
55.67862
54.18219
53.406445
53.0043


In [25]:
print(forward(x,w))

[4.693667  1.387335  5.0810013]


#### Gradient descent using torch

In [43]:
x = torch.tensor([1,2,3],dtype=torch.float32)
y = torch.tensor([1,2,3],dtype=torch.float32)
w = torch.tensor([1,2,3],dtype=torch.float32,requires_grad=True)

def forward(x,w):
    return x*w
def loss(y_pred,y):
    return ((y_pred-y)**2).mean()

optimizer = torch.optim.Adam([w],lr=learning_rate)

In [44]:
learning_rate=0.1

for i in range(10):
    y_pred = forward(x,w)
    loss_value = loss(y_pred,y)
    loss_value.backward()
    optimizer.step()
    w.grad.zero_()
    print(loss_value)


tensor(13.3333, grad_fn=<MeanBackward0>)
tensor(11.9100, grad_fn=<MeanBackward0>)
tensor(10.5760, grad_fn=<MeanBackward0>)
tensor(9.3327, grad_fn=<MeanBackward0>)
tensor(8.1808, grad_fn=<MeanBackward0>)
tensor(7.1207, grad_fn=<MeanBackward0>)
tensor(6.1520, grad_fn=<MeanBackward0>)
tensor(5.2736, grad_fn=<MeanBackward0>)
tensor(4.4837, grad_fn=<MeanBackward0>)
tensor(3.7794, grad_fn=<MeanBackward0>)


In [47]:
import torch
import torch.nn as nn
import torch.nn.functional as F

input_doses = torch.linspace(start=0, end=1, steps=11)

In [48]:
class BasicNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.w00 = nn.Parameter(torch.tensor(1.7), requires_grad=False)
        self.b00 = nn.Parameter(torch.tensor(-0.85), requires_grad=False)
        self.w01 = nn.Parameter(torch.tensor(-40.8), requires_grad=False)
        
        self.w10 = nn.Parameter(torch.tensor(12.6), requires_grad=False)
        self.b10 = nn.Parameter(torch.tensor(0.0), requires_grad=False)
        self.w11 = nn.Parameter(torch.tensor(2.7), requires_grad=False)

        self.final_bias = nn.Parameter(torch.tensor(0.), requires_grad=True) 

    def forward(self,input):
        input_to_top_relu = input * self.w00 + self.b00
        top_relu_output = F.relu(input_to_top_relu)
        scaled_top_relu_output = top_relu_output * self.w01
        
        input_to_bottom_relu = input * self.w10 + self.b10
        bottom_relu_output = F.relu(input_to_bottom_relu)
        scaled_bottom_relu_output = bottom_relu_output * self.w11
    
        input_to_final_relu = scaled_top_relu_output + scaled_bottom_relu_output + self.final_bias
        
        output = F.relu(input_to_final_relu)
        
        return output


In [51]:
model = BasicNN()

inputs = torch.tensor([0., 0.5, 1.])
labels = torch.tensor([0., 1., 0.])

optimizer = torch.optim.Adam(model.parameters(),lr=0.001)


In [55]:
for i in range(10000):
    total_loss = 0
    for y in range(len(inputs)):
        y_pred = model(inputs[y])
        loss = (y_pred-y)**2
        loss.backward()
        total_loss+=float(loss)
    optimizer.step()
    optimizer.zero_grad()
    print(total_loss)


250.84182739257812
250.8106689453125
250.7794952392578
250.7483367919922
250.71717834472656
250.68601989746094
250.65487670898438
250.623779296875
250.59262084960938
250.5614776611328
250.53033447265625
250.49925231933594
250.46810913085938
250.4369659423828
250.40582275390625
250.37474060058594
250.34361267089844
250.31248474121094
250.28140258789062
250.25027465820312
250.21920776367188
250.18807983398438
250.15696716308594
250.1259002685547
250.09478759765625
250.063720703125
250.03260803222656
250.00155639648438
249.97044372558594
249.93939208984375
249.90829467773438
249.8772430419922
249.8461456298828
249.81509399414062
249.7840576171875
249.75296020507812
249.721923828125
249.69088745117188
249.65980529785156
249.62876892089844
249.59774780273438
249.566650390625
249.53562927246094
249.50460815429688
249.47352600097656
249.4425048828125
249.4114990234375
249.38047790527344
249.34947204589844
249.3184051513672
249.28738403320312
249.25637817382812
249.2253875732422
249.1943817138

In [56]:
output_values = model(input_doses)

In [57]:
print(output_values)

tensor([0.0000, 0.0000, 0.0000, 0.6609, 4.0629, 7.4649, 3.9309, 0.3969, 0.0000,
        0.0000, 0.0000], grad_fn=<ReluBackward0>)
